In [2]:
import pandas as pd
import os
import shutil
import numpy as np

from QC.utils import shell_do

In [3]:
# function definitions

def make_gencall_cmds(idat_path, bpm_file, cluster_file, out_path, iaap=None):
    
    cmd = f'\
{iaap} gencall \
{bpm_path} \
{cluster_file} \
{out_path}/ped/ \
-f {idat_path}/{code} \
-p \
-t 8'

In [4]:
basedir = '/data/CARD/PD/GP2/raw_genotypes'
out_genotypes = '/data/CARD/PD/GP2/genotypes'
shulman_ny_path = f'{basedir}/shulman_ny'
sample_info_path = f'{shulman_ny_path}/sample_info'
gtc_file_path = f'{shulman_ny_path}/GP2_GCT_files'
raw_idat_file_path = f'{shulman_ny_path}/GP2_Shulman'
ped_dir = f'{shulman_ny_path}/ped'
plink_dir = f'{shulman_ny_path}/plink'
swarm_scripts_dir = f'swarm_scripts'
idat_dir = f'{shulman_ny_path}/idats'

# create ped and plink directories for raw geno outputs if they don't exist
os.makedirs(ped_dir, exist_ok=True)
os.makedirs(plink_dir, exist_ok=True)
os.makedirs(f'{plink_dir}/indiv_samples', exist_ok=True)
os.makedirs(swarm_scripts_dir, exist_ok=True)
os.makedirs(idat_dir, exist_ok=True)

# files
key_file = f'{sample_info_path}/sample_key.txt'
manifest_txt_path = f'{sample_info_path}/sample_manifest.txt'
bpm = f'{sample_info_path}/NeuroBooster_20042459_A1.bpm'
cluster_file = f'{sample_info_path}/NBSCluster_file_n1393_011921.egt'

# phenos
bmc_pheno_file = f'{sample_info_path}/BCM_20210201_samples.csv'
umd_pheno_file = f'{sample_info_path}/UMD_20210201_samples.csv'

# executables
iaap = 'executables/iaap-cli-linux-x64-1.1.0-sha.80d7e5b3d9c1fdfc2e99b472a90652fd3848bbc7/iaap-cli/iaap-cli'

In [43]:
# some rearranging
# !mkdir {shulman_ny_path}/sample_info
# !cp {gtc_file_path}/Key\ File_FINAL_Shulman_and_NY_011421.txt {sample_info_path}/sample_key.txt
# !cp {gtc_file_path}/FINALSS_after_rerun__Shulman_and_NY_011421.csv {sample_info_path}/sample_manifest.txt
# !cp {gtc_file_path}/NeuroBooster_20042459_A1.bpm {sample_info_path}/
# !cp {gtc_file_path}/NBSCluster_file_n1393_011921.egt {sample_info_path}/



In [7]:
# create updated ID and phenotype files for later
manifest = pd.read_csv(manifest_txt_path, header=10)
bmc_pheno = pd.read_csv(bmc_pheno_file)
umd_pheno = pd.read_csv(umd_pheno_file)
pheno = bmc_pheno.append(umd_pheno)

pheno['Original_clinicalID'] = pheno['Original_clinicalID'].astype(str)
manifest['Sample_ID'] = manifest['Sample_ID'].astype(str)
manifest['filename'] = manifest['SentrixBarcode_A'].astype(str) + '_' + manifest['SentrixPosition_A']
pheno_out = manifest.merge(pheno, how='left', left_on='Sample_ID', right_on='Original_clinicalID')
pheno_out['IID'] = pheno_out.SentrixBarcode_A.astype(str) + '_' + pheno_out.SentrixPosition_A.astype(str)
pheno_out['FID'] = 0
pheno_out['FID_new'] = 0
pheno_out['pheno'] = 0
pheno_out.loc[pheno_out.Phenotype == 'PD', 'pheno'] = 2
pheno_out.loc[pheno_out.Phenotype == 'Control', 'pheno'] = 1
pheno_out.loc[pheno_out.Phenotype == np.nan, 'pheno'] = 0

pheno_out[['FID','IID', 'FID_new', 'Sample_ID']].to_csv(f'{sample_info_path}/update_ids.txt', sep='\t', header=False, index=False)
pheno_out[['FID_new', 'Sample_ID', 'pheno']].to_csv(f'{sample_info_path}/update_pheno.txt', sep='\t', header=False, index=False)

In [11]:
# create a folder in idats for each plate in new idat_dir
for code in manifest.SentrixBarcode_A.unique():
    if os.path.exists(f'{idat_dir}/{code}'):
        print(f'{idat_dir}/{code} already exists')
    else:
        os.mkdir(f'{idat_dir}/{code}')

# copy idat intensity files to respective directories under idat_dir
missing_idats = []

for i, filename in enumerate(manifest.filename):
    sentrix_code = manifest.SentrixBarcode_A.iloc[i]
    grn = f'{raw_idat_file_path}/{sentrix_code}/{filename}_Grn.idat'
    red = f'{raw_idat_file_path}/{sentrix_code}/{filename}_Red.idat'

    if os.path.isfile(grn):
        shutil.copyfile(src=grn, dst=f'{idat_dir}/{sentrix_code}/{filename}_Grn.idat')
    else:
        missing_idats.append(grn)

    if os.path.isfile(red):
        shutil.copyfile(src=red, dst=f'{idat_dir}/{sentrix_code}/{filename}_Red.idat')
    else:
        missing_idats.append(red)

len(missing_idats)

0

In [44]:
with open(f'{swarm_scripts_dir}/idat_to_ped.swarm', 'w') as f:
    
    for code in manifest.SentrixBarcode_A.unique():
        
        idat_to_ped_cmd = f'\
{iaap} gencall \
{bpm} \
{cluster_file} \
{ped_dir}/ \
-f {idat_dir}/{code} \
-p \
-t 8'
        
        f.write(f'{idat_to_ped_cmd}\n')
f.close()

In [45]:
# !swarm -f {swarm_scripts_dir}/idat_to_ped.swarm -g 32 -t 16 --time=10:00:00 --logdir swarm --gres=lscratch:20 --partition=norm

12454654


In [46]:
# copy map file to match name of each ped
map = f'{ped_dir}/NeuroBooster_20042459_A1.map'
for filename in manifest.filename:
    ped = f'{ped_dir}/{filename}.ped'
    out_map = f'{ped_dir}/{filename}.map'
    if os.path.isfile(ped):
        shutil.copyfile(src=map, dst=out_map)
    else:
        print(f'{ped} does not exist!')
        print(f'{out_map} creation cancelled')


In [53]:

with open(f'{ped_dir}/make_bed.swarm', 'w') as f:
    for filename in manifest.filename:
        ped = f'{ped_dir}/{filename}'
        make_bed_cmd = f'\
plink \
--file {ped} \
--make-bed \
--out {plink_dir}/indiv_samples/{filename}'

        f.write(f'{make_bed_cmd}\n')
f.close()


        # shell_do(make_bed_cmd)


In [54]:
!swarm -f {ped_dir}/make_bed.swarm -g 8 -t 8 --time=10:00:00 --logdir swarm --gres=lscratch:20 --partition=norm

12455471


In [56]:
# write plink merge command
with open(f"{plink_dir}/merge_bed.list", 'w') as f:
    for filename in manifest.filename:
        bed = f'{plink_dir}/indiv_samples/{filename}'
        f.write(f'{bed}\n')
f.close()

with open(f"{plink_dir}/merge.swarm", 'w') as f:

    plink_merge_cmd = f'\
plink \
--merge-list {plink_dir}/merge_bed.list \
--update-ids {sample_info_path}/update_ids.txt \
--make-bed \
--out {plink_dir}/indiv_samples/shulman_merge'
    f.write(f"{plink_merge_cmd}")
f.close()

In [58]:
!swarm -f {plink_dir}/merge.swarm -g 64 -t 32 --time=10:00:00 --logdir swarm --gres=lscratch:20 --partition=norm

12456657


In [59]:
!plink --bfile {plink_dir}/indiv_samples/shulman_merge --pheno {sample_info_path}/update_pheno.txt --make-bed --out {plink_dir}/shulman

PLINK v1.90b4.4 64-bit (21 May 2017)           www.cog-genomics.org/plink/1.9/
(C) 2005-2017 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /data/CARD/PD/GP2/raw_genotypes/shulman_ny/plink/shulman.log.
Options in effect:
  --bfile /data/CARD/PD/GP2/raw_genotypes/shulman_ny/plink/indiv_samples/shulman_merge
  --make-bed
  --out /data/CARD/PD/GP2/raw_genotypes/shulman_ny/plink/shulman
  --pheno /data/CARD/PD/GP2/raw_genotypes/shulman_ny/sample_info/update_pheno.txt

1547809 MB RAM detected; reserving 773904 MB for main workspace.
2004347 variants loaded from .bim file.
1393 people (765 males, 601 females, 27 ambiguous) loaded from .fam.
Ambiguous sex IDs written to
/data/CARD/PD/GP2/raw_genotypes/shulman_ny/plink/shulman.nosex .
1285 phenotype values present after --pheno.
phenotypes to be ignored, use the --allow-no-sex flag.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 1393 founders and 0 nonfounders present.
Calculat

In [3]:
# call QC pipeline

with open(f"{plink_dir}/run_qc_pipeline.swarm", 'w') as f:
    
    qc_pipeline_cmd = 'python3 run_qc_pipeline.py --geno /data/CARD/PD/GP2/raw_genotypes/shulman_ny/plink/shulman --ref /data/LNG/vitaled2/1kgenomes/1kg_ashkj_ref_panel_gp2_pruned --ref_labels /data/LNG/vitaled2/1kgenomes/ref_panel_ancestry.txt --out /data/CARD/PD/GP2/raw_genotypes/shulman_ny/plink/shulman'

    f.write(f'{qc_pipeline_cmd}')
f.close()





In [5]:
!swarm -f {plink_dir}/run_qc_pipeline.swarm -g 64 -t 64 --time=24:00:00 --logdir {plink_dir}/swarm --gres=lscratch:20 --module python/3.7,perl,GCTA,plink,flashpca,vcftools,bcftools --partition=norm

12630566


In [6]:
print(f'swarm -f {plink_dir}/run_qc_pipeline.swarm -g 64 -t 64 --time=24:00:00 --logdir {plink_dir}/swarm --gres=lscratch:20 --module python/3.7,perl,GCTA,plink,flashpca,vcftools,bcftools --partition=norm')

swarm -f /data/CARD/PD/GP2/raw_genotypes/shulman_ny/plink/run_qc_pipeline.swarm -g 64 -t 64 --time=24:00:00 --logdir /data/CARD/PD/GP2/raw_genotypes/shulman_ny/plink/swarm --gres=lscratch:20 --module python/3.7,perl,GCTA,plink,flashpca,vcftools,bcftools --partition=norm


In [ ]:
# get basic statistics
# plink_miss_cmd = f'\
# plink \
# --bfile {shulman_out}/plink/shulman \
# --missing \
# --out {shulman_out}/plink/shulman'

# shell_do(plink_miss_cmd)

In [ ]:
# get average call rate
# lmiss = pd.read_csv(f'{shulman_out}/plink/shulman.lmiss', sep='\s+')
# imiss = pd.read_csv(f'{shulman_out}/plink/shulman.imiss', sep='\s+')
# avg_call_rate = 100-lmiss.F_MISS.mean()
# avg_geno_rate = 100-imiss.F_MISS.mean()
# print(f'Average Call Rate: {avg_call_rate}')
# print(f'Average Genotyping Rate: {avg_geno_rate}')